In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/flywire

In [ ]:
import numpy as np
import scipy as sp
import cupy as cp
from utility import M,F, benchmark_mapping, get_best_connected, invert, score, get_highest_out_degrees
rng = np.random.default_rng(0)

In [1]:
def make_best_swap(G,H, mapping, u, scores_by_vertex):

    m = G.shape[0]
    n = H.shape[0]

    row = G[u,:]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = G[:,u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    X = H[mapping[:,None],mapping[None,rnz]].reshape((n,len(rnz)))
    Y = H[mapping[cnz,None],mapping[None,:]].reshape((len(cnz),n))

    u_scores = cp.minimum(row[None,:],X).sum(axis=1) + cp.minimum(col[:,None],Y).sum(axis=0)

    f_u = mapping[u]

    row = H[f_u,mapping]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = H[mapping,f_u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    X = G[:,rnz[None,:]].reshape((m,len(rnz)))
    Y = G[cnz[:,None],:].reshape((len(cnz),m))

    f_u_scores = cp.minimum(row[None,:],X).sum(axis=1) + cp.minimum(col[:,None],Y).sum(axis=0)

    scores = u_scores + f_u_scores + np.minimum(G[u,:],H[mapping,f_u])+np.minimum(G[:,u],H[f_u,mapping])
    scores -= scores_by_vertex
    scores -= scores_by_vertex[u]
    #The actual change will be 2*scores[w], but this does not change which vertex we will choose for the swap.

    w = scores.argmax()
    if scores[w] > 0:

        f_w = mapping[w]

        scores_by_vertex += cp.minimum(G[:,u],H[mapping,f_w])+cp.minimum(G[u,:],H[f_w,mapping])+cp.minimum(G[:,w],H[mapping,f_u])+cp.minimum(G[w,:],H[f_u,mapping])
        scores_by_vertex -= cp.minimum(G[:,u],H[mapping,f_u])+cp.minimum(G[u,:],H[f_u,mapping])+cp.minimum(G[:,w],H[mapping,f_w])+cp.minimum(G[w,:],H[f_w,mapping])

        scores_by_vertex[u] = u_scores[w]+ cp.minimum(G[u,w],H[f_w,f_u])+cp.minimum(G[w,u],H[f_u,f_w])
        scores_by_vertex[w] = f_u_scores[w]+ cp.minimum(G[u,w],H[f_w,f_u])+cp.minimum(G[w,u],H[f_u,f_w])

        mapping[u] = f_w
        mapping[w] = f_u

        return scores[w]
    
    else:
        return 0
    
def get_scores_by_vertex(G,H,mapping):

    m = G.shape[0]
    scores = cp.minimum(G,H[mapping[:,None],mapping[None,:]])
    return scores.sum(axis=0)+scores.sum(axis=1)

Hello World


In [ ]:
A=cp.asarray(M.todense())
B=cp.asarray(F.todense())
N=A.shape[0]
A[np.arange(N),np.arange(N)]=0
B[np.arange(N),np.arange(N)]=0

In [ ]:
G= B
H=A
mapping = cp.asarray(invert(benchmark_mapping))

In [ ]:
scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=scores_by_vertex.sum()//2

for i in range(5):
  for u in cp.arange(N):#range(1319,N):
    improvement = make_best_swap(G,H,mapping,u,scores_by_vertex)
    score_track += improvement
    if u%20 == 0:
      print(f"{i=}, {u=}, {score_track=}",end="\n")